# Exploración de datos con pandas: Adult Income Dataset

En este notebook vamos a:

1. Cargar un dataset real directamente desde la web.
2. Entender su estructura (filas, columnas, tipos de dato).
3. Explorar problemas de calidad de datos (valores faltantes, categorías "raras", outliers).
4. Identificar posibles fuentes de sesgo y variables sensibles.

Dataset: *Adult Income* (censo de EE. UU.). La tarea típica es predecir si el ingreso de una persona es > 50K USD/año.

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", 50)

In [ ]:
url = "https://raw.githubusercontent.com/DataResponsibly/Datasets/master/AdultIncomeData/adult.csv"
df = pd.read_csv(url)

df.head()


In [ ]:
cat_cols = ["workclass", "education", "marital-status", "occupation",
            "relationship", "race", "sex", "native-country", "income"]

for col in cat_cols:
    print(f"\n==== {col} ====")
    print(df[col].value_counts(dropna=False).head(15))

In [ ]:
df.workclass.unique()

In [ ]:
# Conteo de "?" por columna
(df == " ?").sum().sort_values(ascending=False)

In [ ]:
# Reemplazar "?" por NaN
import numpy as np

df_clean = df.replace(" ?", np.nan)

# Porcentaje de valores faltantes por columna
missing_pct = df_clean.isna().mean().sort_values(ascending=False) * 100
missing_pct


### Preguntas

1. ¿En qué variables hay más valores faltantes (NaN / “?”)?
2. ¿Qué tipo de información falta (ocupación, país, etc.)?  
3. ¿Cómo podría afectar esto a un modelo de predicción de ingreso?
4. Si eliminara todas las filas con datos faltantes, ¿a quién estaría dejando por fuera?


In [ ]:
# Versión "agresiva": eliminar filas con algún NaN
df_dropna = df_clean.dropna()

df.shape, df_dropna.shape

In [ ]:
# Comparar distribución de algunas variables antes y después
cols_check = ["sex", "race", "native-country", "occupation"]

for col in cols_check:
    print(f"\n=== {col} - original ===")
    print(df_clean[col].value_counts(normalize=True).head(10))

    print(f"\n=== {col} - dropna ===")
    print(df_dropna[col].value_counts(normalize=True).head(10))


### Sesgos introducidos por decisiones de limpieza

- Al eliminar todas las filas con valores faltantes:
  - ¿Cambió la proporción de mujeres vs. hombres?
  - ¿Cambió la proporción de ciertas razas o países de origen?
  - ¿Qué grupos parecen perder más registros?

Piense como consultor/a:

- ¿Cómo explicaría a un cliente (banco, gobierno) que una decisión de limpieza puede **empeorar** la equidad del sistema?
- ¿Qué alternativas podría explorar en lugar de simplemente borrar filas?


In [ ]:
# Ejemplo: distribución de ingreso por sexo
pd.crosstab(df_dropna["sex"], df_dropna["income"], normalize="index") * 100


In [ ]:
# Distribución de ingreso por raza
pd.crosstab(df_dropna["race"], df_dropna["income"], normalize="index") * 100


## Visualización

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Reemplazar "?" por NaN para facilitar análisis
df = df.replace("?", np.nan)

# Asegurar que la columna de ingreso no tenga espacios
df["income"] = df["income"].str.strip()

# Crear una variable binaria de ingresos altos
df["high_income"] = (df["income"] == ">50K").astype(int)

df[["income", "high_income"]].head()

## 1. Distribuciones: ¿cómo luce nuestra población?

Primero vamos a mirar cómo se distribuyen algunas variables numéricas:

- `age` (edad)
- `hours-per-week` (horas trabajadas por semana)

Usaremos histogramas para:

- Ver si las distribuciones son simétricas o sesgadas.
- Identificar valores extremos (outliers).
- Empezar a pensar qué resumen (media, mediana) tiene más sentido.


In [ ]:
# Histograma de edades
df["age"].hist(bins=30)
plt.xlabel("Edad")
plt.ylabel("Número de personas")
plt.title("Distribución de la edad")
plt.show()


In [ ]:
# Histograma de horas trabajadas por semana
df["hours-per-week"].hist(bins=30)
plt.xlabel("Horas trabajadas por semana")
plt.ylabel("Número de personas")
plt.title("Distribución de horas trabajadas")
plt.show()


### Preguntas de interpretación

1. ¿La distribución de edades es simétrica o está sesgada hacia algún lado?
2. ¿Hay valores extremos (por ejemplo, muchas personas trabajando muy pocas o muchas horas)?
3. Si tuvieras que resumir cada variable en **un solo número**, ¿usaría media, mediana o ambas? ¿Por qué?



In [ ]:
summary = pd.DataFrame({
    "media": df[["age", "hours-per-week"]].mean(),
    "mediana": df[["age", "hours-per-week"]].median()
})
summary


**Ejercicio:**

Redacte una frase para cada variable que combine número + interpretación, por ejemplo:

- “La mediana de edad en el dataset es de **X años**, lo que indica que la mitad de la población tiene menos de esa edad.”
- “La media de horas trabajadas por semana es de **Y horas**, pero el histograma muestra que hay un grupo no despreciable trabajando muchas más horas que el promedio.”

La idea es practicar cómo pasamos de números a frases que cualquier persona pueda entender.


## 2. Comparando grupos: ¿quién tiene más probabilidad de tener ingresos altos?

Vamos a calcular y visualizar la **proporción de personas con ingresos >50K** según:

- Sexo (`sex`)
- Raza (`race`)
- Nivel educativo (`education`)

Esto nos sirve para:

- Ver brechas entre grupos.
- Empezar a imaginar qué pasaría si entrenáramos un modelo sin cuidar estos temas.

In [ ]:
def tasa_high_income_por_grupo(df, col_grupo):
    """
    Calcula la proporción de personas con high_income = 1 por categoría de col_grupo.
    Devuelve un DataFrame con columnas [col_grupo, "tasa_high_income"].
    """
    tabla = df.groupby(col_grupo)["high_income"].mean().reset_index()
    tabla = tabla.sort_values("high_income", ascending=False)
    tabla.rename(columns={"high_income": "tasa_high_income"}, inplace=True)
    return tabla

tasa_sex = tasa_high_income_por_grupo(df, "sex")
tasa_race = tasa_high_income_por_grupo(df, "race")
tasa_edu = tasa_high_income_por_grupo(df, "education")

tasa_sex


In [ ]:
# Sexo
tasa_sex.plot(kind="bar", x="sex", y="tasa_high_income", legend=False)
plt.ylabel("Proporción con ingreso >50K")
plt.title("Ingreso alto por sexo")
plt.ylim(0, 1)
plt.show()

# Raza
tasa_race.plot(kind="bar", x="race", y="tasa_high_income", legend=False)
plt.ylabel("Proporción con ingreso >50K")
plt.title("Ingreso alto por raza")
plt.ylim(0, 1)
plt.show()

# Educación (puede ser muchas categorías; muestra las principales)
tasa_edu.head(10).plot(kind="bar", x="education", y="tasa_high_income", legend=False)
plt.ylabel("Proporción con ingreso >50K")
plt.title("Ingreso alto por nivel educativo (top 10 categorías)")
plt.ylim(0, 1)
plt.show()


In [ ]:
# Distribución conjunta de income por sexo
crosstab_sex = pd.crosstab(df["sex"], df["income"], normalize="index") * 100
crosstab_sex


In [ ]:
crosstab_sex.plot(kind="bar", stacked=True)
plt.ylabel("% dentro de cada sexo")
plt.title("Distribución de ingresos por sexo")
plt.show()


## Otros gráficos

In [ ]:
# Distribución de income (<=50K vs >50K)
income_counts = df["income"].value_counts()

plt.figure()
plt.pie(
    income_counts,
    labels=income_counts.index,
    autopct="%1.1f%%",
    startangle=90
)
plt.title("Distribución de niveles de ingreso")
plt.show()


## Scatterplot con tres dimensiones

Un diagrama de dispersión (scatterplot) muestra la relación entre dos variables numéricas.
Podemos añadir una tercera dimensión usando:

- Color (para distinguir grupos).
- Tamaño de los puntos (para indicar intensidad o magnitud).

Ejemplo: edad (x), horas trabajadas (y), color según si la persona tiene ingreso >50K.


In [ ]:
import seaborn as sns

In [ ]:
# Muestra aleatoria para que el gráfico no quede tan saturado
df_sample = df.sample(n=3000, random_state=42)

plt.figure(figsize=(7, 6))
sns.scatterplot(
    data=df_sample,
    x="age",
    y="hours-per-week",
    hue="sex",
    alpha=0.6
)
plt.title("Edad vs horas trabajadas por semana, coloreado por sexo")
plt.xlabel("Edad")
plt.ylabel("Horas trabajadas por semana")
plt.legend(title="Sexo")
plt.show()


In [ ]:
df_sample = df.sample(n=3000, random_state=42).copy()
df_sample["capital-gain"] = df_sample["capital-gain"].fillna(0)

plt.figure(figsize=(7, 6))
sns.scatterplot(
    data=df_sample,
    x="age",
    y="hours-per-week",
    hue="sex",
    size="capital-gain",
    alpha=0.6,
    sizes=(10, 80)  # rango de tamaños
)
plt.title("Edad vs horas trabajadas, color por sexo, tamaño por capital-gain")
plt.xlabel("Edad")
plt.ylabel("Horas trabajadas por semana")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()


### ¿Qué podemos leer de estos scatterplots?

- ¿En qué zonas del gráfico se concentran las personas con ingresos altos (>50K)?
- ¿Hay diferencias claras por sexo en la relación edad–horas trabajadas?
- Cuando añadimos el tamaño según `capital-gain`, ¿aparecen patrones nuevos o solo ruido visual?
- Si entrenáramos un modelo de IA usando estas variables, ¿qué tipos de decisiones podría aprender?
- ¿Hay riesgos de que el modelo favorezca ciertos rangos de edad, sexo o patrones de trabajo?
